In [2]:
import json
with open('/Users/siblose/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# import the YelpAPI Class
from yelpapi import YelpAPI
# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [ ]:
#To use the "businesses search" endpoint,  we will use the yelp_api.search_query method.

In [4]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [ ]:
#The only arguments/parameters that the docstring mentions are location or latitude & longitude, because the others are optional
#but it will accept ANY of the parameters that are accepted by the business_search endpoint.

In [5]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location='NY, NY',
                                       term='Pizza')
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [6]:
search_results['total']

12300

In [8]:
import pandas as pd

In [9]:
biz = pd.DataFrame(search_results['businesses'])
biz.head(2) 

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5084,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/od36nF...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2703,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844


In [ ]:
#Pagination and "offset"
#Total Results vs. Businesses

#If we check the total number of results, we can see that we had 12,000 businesses that met our search criteria

In [10]:
## total number of matching businesses
search_results['total']

12300

In [11]:
#However, if we check our list of businesses, we will see that we only received data for 20 businesses.
## how many businesses in our results
len(search_results['businesses'])

20

In [ ]:
#The Yelp API will only return a "page" of 20 results at a time.
#If we want to get the next page of results, 
#we will perform another API call, but we will add an additional argument called "offset."

In [12]:
# add offset to our original api call
search_results = yelp_api.search_query(location='NY, NY',
                                       term='Pizza',
                                       offset = 20)

In [13]:
biz20 = pd.DataFrame(search_results['businesses'])
biz20.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,vpkTNjq9dRh9rT3Nh0pe-A,nolita-pizza-new-york-2,Nolita Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/_BKVVo...,False,https://www.yelp.com/biz/nolita-pizza-new-york...,226,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.7208883, 'longitude': -73.9962...","[pickup, delivery]",$,"{'address1': '68 Kenmare St', 'address2': None...",+16468959131,(646) 895-9131,1725.054309
1,Q9F2ocrmYuGt1yn3M7MOBw,lucali-brooklyn-3,Lucali,https://s3-media1.fl.yelpcdn.com/bphoto/rMlNCZ...,False,https://www.yelp.com/biz/lucali-brooklyn-3?adj...,2012,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.681840746711224, 'longitude': ...",[delivery],$$,"{'address1': '575 Henry St', 'address2': None,...",+17188584086,(718) 858-4086,2673.589406


In [ ]:
#Above should have different results than your original call when you make the offset call.
#Now you can combine the results (so far) into one dataframe using pd.concat()


In [14]:
## concatenate the previous results and new results. 
businesses = pd.concat([biz, biz20],
                      ignore_index=True)
display(businesses.head(3), businesses.tail(3))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5084,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/od36nF...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2703,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3193,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
37,l6yVO8l8E5XI9ArgOy5rgw,ltd-pizza-and-bar-new-york,LTD Pizza and Bar,https://s3-media2.fl.yelpcdn.com/bphoto/Lt0xvq...,False,https://www.yelp.com/biz/ltd-pizza-and-bar-new...,32,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.724242, 'longitude': -74.00802}","[restaurant_reservation, pickup, delivery]",NaN,"{'address1': '225 Hudson St', 'address2': None...",+12124191618,(212) 419-1618,2391.701096
38,yH6t1JWcwkdWG4CS5KPvow,adriennes-pizzabar-new-york,Adrienne's Pizzabar,https://s3-media3.fl.yelpcdn.com/bphoto/YePG9J...,False,https://www.yelp.com/biz/adriennes-pizzabar-ne...,1850,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.70424819999999, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '54 Stone St', 'address2': None, ...",+12122483838,(212) 248-3838,1340.117592
39,pfccCt5bZaGv8U1J8oFNjQ,siena-pizza-and-cannoli-new-york-2,Siena Pizza & Cannoli,https://s3-media4.fl.yelpcdn.com/bphoto/TRQ_bK...,False,https://www.yelp.com/biz/siena-pizza-and-canno...,134,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.708152, 'longitude': -74.013864}","[pickup, delivery]",$,"{'address1': '14 Rector St', 'address2': None,...",+12124060918,(212) 406-0918,1661.278325


In [ ]:
#done